In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [16]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/main/day29-sklearn-pipelines/train.csv')

In [17]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [18]:
df = df.drop(['PassengerId','Name','Cabin','Ticket'], axis = 1)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['Survived'], axis = 1), df['Survived'], test_size = 0.2, random_state = 42)

In [20]:
x_train.shape, y_train.shape

((712, 7), (712,))

In [21]:
trf1 = ColumnTransformer([('impute_age', SimpleImputer(),[2]),
                          ('impute_embark', SimpleImputer(strategy = 'most_frequent'),[6])],
                         remainder = 'passthrough')

It is a good practice to use index numbers instead of name of the column because output of `Columntransformer` is array which doesnot have column name and can cause problems.

In [22]:
trf2 = ColumnTransformer([('ohe_sex_embarked', OneHotEncoder(sparse = False, handle_unknown = 'ignore'),[1,6])]
                         , remainder = 'passthrough')

In [23]:
trf3 = ColumnTransformer([('scale', MinMaxScaler(), slice(0,10))])

In [30]:
trf4 = SelectKBest(score_func = chi2, k = 8)

In [31]:
trf5 = DecisionTreeClassifier()

In [32]:
pipeline = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
]   
)

# **Pipeline VS make_pipelines**
Pipeline requires extra step as name of pipeline but make_pipeline doesnot

In [33]:
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [34]:
from sklearn import set_config
set_config(display = 'diagram')

In [35]:
pipe.fit(x_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embark',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7f750edacb80>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [36]:
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embark',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x7f750edacb80>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [46]:
pipe.named_steps['columntransformer-1'].transformers_[0][1].statistics_

array([29.49884615])

In [45]:
pipe.named_steps['columntransformer-1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [47]:
y_pred = pipe.predict(x_test)

In [48]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

0.6256983240223464

# **CROSS VALIDATION PIPELINE**

In [49]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, x_train, y_train, cv = 5, scoring = 'accuracy').mean()

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` 

0.6391214419383433

# Grid_Search using pipeline

In [61]:
import warnings
warnings.filterwarnings('ignore')

In [58]:
params = {
    'decisiontreeclassifier__max_depth' :[1,2,3,4,5,None]
}

In [62]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv = 5, scoring = 'accuracy')
grid.fit(x_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer-1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embark',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('columntransformer-2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('columntransformer-3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('selectkbest',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7f750edacb80>)),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5,
                                                               None]},
             scoring='accuracy')

In [63]:
grid.best_score_

0.6391214419383433

In [66]:
grid.best_params_

{'decisiontreeclassifier__max_depth': 2}

In [65]:
import pickle

# **Exporting the Pipleline**

In [68]:
pickle.dump(pipe, open('pipe.pkl', 'wb'))